In [9]:
import random
import sys
import time
from typing import List, Optional

from pyblend.algorithm.constructive import Constructive, LinModel, PostModel, PreModel
from pyblend.algorithm.heuristic import Heuristic, LAHC, SA
from pyblend.algorithm.neighborhood import (Shift, SimpleSwap, SmartShift,
                                            SmartSimpleSwap, SmartSwap, SmartSwitch,
                                            Swap, Switch)
from pyblend.config import Objective, Parameters
from pyblend.model.problem import Problem
from pyblend.model.solution import Solution
from pyblend.utils import handle_input_path, handle_output_path
from pyblend.main import feedback_approach, solve, construct, read_args



In [10]:
start_time = time.time()
parms: Parameters = {
    "constructive": "postmodel",
    "algorithm": "",
    "feedback": 0,
    "seed": 0,
    "maxiters": int(1e3),
    "lsize": int(1e3),
    "alpha": 0.9,
    "samax": int(1e3),
    "t0": 1.0,
}

In [11]:
random.seed(parms["seed"])

input_path = handle_input_path("instance_1.json")
output_path = handle_output_path("../out/json/out_1-2.json")

In [12]:
problem: Problem = Problem(input_path)

In [19]:
problem.outputs[0]

id: 1
destination: 1
weight: 300000
quality: [parameter: Fe
minimum: 57
maximum: 100
goal: 57
importance: 10
, parameter: SiO2
minimum: 0
maximum: 5.8
goal: 5.8
importance: 1000
, parameter: Al2O3
minimum: 0
maximum: 4.9
goal: 4.9
importance: 100
, parameter: P
minimum: 0
maximum: 0.07
goal: 0.07
importance: 100
, parameter: +31.5
minimum: 0
maximum: 10
goal: 10
importance: 100
, parameter: -6.3
minimum: 0
maximum: 25
goal: 25
importance: 100
]
time: 4

In [20]:
parms["constructive"]

'postmodel'

In [21]:
solution: Solution = Solution(problem)

In [23]:
problem.info[1:]

[1000, 1]

In [24]:
model: LinModel = LinModel(problem)

In [25]:
constructive: Constructive = construct(problem, solution, model, parms)


Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 28 2021 

Starting solution of the Linear programming problem using Primal Simplex



In [31]:
constructive.solution.work_time(1)

(0.06, 161.76)

In [36]:
constructive._problem.outputs[0].weight / sum([
                eng.speed_reclaim for eng in constructive.problem.engines
            ])

115.38461538461539

In [ ]:
solver: Optional[Heuristic] = None
if parms["algorithm"] != "":
    solver = solve(problem, solution, constructive, parms)
    solution = solver.best_solution

if parms["feedback"] > 0:
    feedback_approach(solution, model, solver, constructive, parms)

solution.set_deliveries()
solution.write(output_path, round(time.time()-start_time, 2))